In [8]:
#check if board is solved
def is_solved(board):
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] == 0:
                return False
    return True

#check if there is only one option within the possible_matrix
def easy_move(board, possible_matrix):
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] == 0 and len(possible_matrix[x][y]) == 1:
                return True, x, y, possible_matrix[x][y][0]
    return False, 0, 0, 0

#update possible matrix
def update_possible_matrix(vert, hori, number, possible_matrix):
    for i in range(len(possible_matrix)):
        if number in possible_matrix[i][hori]:
            possible_matrix[i][hori].remove(number)
        if number in possible_matrix[vert][i]:
            possible_matrix[vert][i].remove(number)

#     if hori == vert:
#         for i in range(len(possible_matrix)):
#             if number in possible_matrix[i][i]:
#                 possible_matrix[i][i].remove(number)
#     if hori + vert == len(possible_matrix) - 1:
#         for i in range(len(possible_matrix)):
#             if number in possible_matrix[i][len(possible_matrix) - 1 - i]:
#                 possible_matrix[i][len(possible_matrix) - 1 - i].remove(number)
    
    #only works with 9x9
    base_vertical = vert // 3 * 3
    base_horizontal = hori // 3 * 3
    
    for v in range(base_vertical, base_vertical + 3):
        for h in range(base_horizontal, base_horizontal + 3):
            if number in possible_matrix[v][h]:
                possible_matrix[v][h].remove(number)
                
    possible_matrix[vert][hori] = [number]
    
    return possible_matrix

def initialize_possible_matrix(board):
    digits = list(range(1, len(board) + 1))
    possible_matrix = [[[x for x in digits]for x in digits]for x in digits]
    
    for x in range(len(board)):
        for y in range(len(board)):
            if board[x][y] != 0:
                possible_matrix = update_possible_matrix(x, y, board[x][y], possible_matrix)
    
    return possible_matrix

#print board
def print_board(board):
    for row in board:
        print(row)

def print_possible_matrix(possible_matrix):
    print("This is the possible matrix")
    for x in range(len(possible_matrix)):
        for y in range(len(possible_matrix)):
            print("{} {}".format(x, y), end = " ")
            print(possible_matrix[x][y])

# Takes a 81-digit string and converts it into a 9x9 array
def stringToBoard(s):
    if not(s.isnumeric() and len(s) == 81):
      print("Invalid string! Must be an 81-digit string of numbers")
      return
    board = []
    for i in range(9):
        row = []
        for j in range(9):
            row.append(int(s[9*i + j]))
        board.append(row)
    return board'

def sudoku_solver(board):
    
    return board

#main
# List of board strings:
#http://forum.enjoysudoku.com/patterns-game-results-t6291.html
# empty board = "000000000000000000000000000000000000000000000000000000000000000000000000000000000"
original_board = stringToBoard("926781003800439620403026000080000070010000005034058060107060452060007000002105080")
board = stringToBoard("926781003800439620403026000080000070010000005034058060107060452060007000002105080")

possible_matrix = initialize_possible_matrix(board)

solved = is_solved(board)
while not solved:    
    easy_possible, vert, hori, num = easy_move(board, possible_matrix)
    if easy_possible:
        board[vert][hori] = num
        possible_matrix = update_possible_matrix(vert, hori, num, possible_matrix)
        #ai stuff
    else:
        # guess function
        # manual move
        # later is guess move
        
        vert = int(input("Insert vertical index"))
        hori = int(input("Insert horizontal index"))
        num = int(input("Insert number"))
        
        board[vert][hori] = num
    
    solved = is_solved(board)

print("FINISHED")
print("This is the original board")
print_board(original_board)
print("This is the finished board")
print_board(board)

FINISHED
This is the original board
[9, 2, 6, 7, 8, 1, 0, 0, 3]
[8, 0, 0, 4, 3, 9, 6, 2, 0]
[4, 0, 3, 0, 2, 6, 0, 0, 0]
[0, 8, 0, 0, 0, 0, 0, 7, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 5]
[0, 3, 4, 0, 5, 8, 0, 6, 0]
[1, 0, 7, 0, 6, 0, 4, 5, 2]
[0, 6, 0, 0, 0, 7, 0, 0, 0]
[0, 0, 2, 1, 0, 5, 0, 8, 0]
This is the finished board
[9, 2, 6, 7, 8, 1, 5, 4, 3]
[8, 5, 1, 4, 3, 9, 6, 2, 7]
[4, 7, 3, 5, 2, 6, 1, 9, 8]
[6, 8, 5, 3, 1, 2, 9, 7, 4]
[2, 1, 9, 6, 7, 4, 8, 3, 5]
[7, 3, 4, 9, 5, 8, 2, 6, 1]
[1, 9, 7, 8, 6, 3, 4, 5, 2]
[5, 6, 8, 2, 4, 7, 3, 1, 9]
[3, 4, 2, 1, 9, 5, 7, 8, 6]
